In [ ]:
pip list

In [ ]:
import pandas as pd
import urllib.request as urlr
import numpy as np
import os

In [ ]:
%time
"""
Open chosen dataset from current directory. Otherwise download and open
library - storage of dataset links
filedata - dataset
"""
library ={1:'https://criteo-bucket.s3.eu-central-1.amazonaws.com/criteo.csv.gz',
          2: None}
urlfile = library[1]
rfile = urlr.urlopen(urlfile)
print('Server status code: ',[rfile.getcode()])
file = os.path.basename(rfile.url)
os.path.join(os.getcwd(), os.path.basename(rfile.url))
if file not in os.listdir(os.getcwd()):
    print('downloading...', end='')
    urlr.urlretrieve(urlfile, os.path.join(os.getcwd(), os.path.basename(rfile.url)))
    print('success')
filedata = pd.read_csv(os.path.basename(rfile.url), nrows=10**4, compression='gzip', error_bad_lines=False)
filedata

In [ ]:
"""
Sorting a column
x - list
indx - column to sort
"""

def Sort(x, indx):
    coerc = x
    n = len(x) - 1

    for i in range(n):
        if coerc[i][indx] > coerc[i+1][indx]:
            temp = coerc[i]
            coerc[i] = coerc[i+1]
            coerc[i+1] = temp
        if i == n-1 and n != 0:
            coerc[:n] = Sort(coerc[:n], indx)
    return(coerc)
"""
Disjointing statistic conteied in dictionary
All - dictionary
Coerc - storage for disjointed statistic
"""
def Processing(All):
    Coerc = [None]*len(All)
    
    for i in range(len(Coerc)):
        Coerc[i] = np.append(list(All.keys())[i], list(All.values())[i])

    for i in range(len(Coerc)):
        Coerc[i][4:16] /= Coerc[i][40]

    for j in range(4, 12):
        Coerc = Sort(Coerc, j)
        for i in range(len(Coerc)-1):
            if Coerc[i][j+24] > Coerc[i+1][j+12]:
                contest = Coerc[i][j+24] - Coerc[i+1][j+12]
                balance = contest * Coerc[i][40]/(Coerc[i][40]+Coerc[i+1][40])
                Coerc[i+1][j+12] -= balance
                Coerc[i][j+24] -= contest - balance
    return(Coerc)

In [ ]:
%%time
"""
Writing a features of class in dictionary
"""
All = {}

for i, data in filedata.iterrows():
    index = tuple(data.values[12:])
    data_values = data.values[:12]
    if index in All:
        All[index][12:24] = np.minimum(All[index][12:24], data_values)
        All[index][24:36] = np.maximum(All[index][24:36], data_values)
        All[index][36]   += 1
        All[index][00:12] += data_values
    else:
        All[index]  = np.append([data_values]*3, [1])

print(Processing(All), sep='\n')